In [ ]:
import descriptors
from rdkit import Chem
from rdkit.Chem.EState.Fingerprinter import FingerprintMol
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit.Chem.rdmolops import RDKFingerprint
import pandas as pd
import numpy as np
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.model_selection import KFold, ShuffleSplit, GridSearchCV, cross_val_score, cross_validate
from sklearn.linear_model import Lasso, LinearRegression
import os
import glob
from scipy.stats import norm
import math
import random

data = pd.read_excel('initial dataset.xlsx')
data['Mols'] = data['SMILES'].apply(Chem.MolFromSmiles)
data['Mols'] = data['Mols'].apply(Chem.AddHs)

bond_types,x_SOB = literal_bag_of_bonds(list(data['Mols']))
#x_ecfp = descriptors.ECFP_fingerprints(list(data['Mols']))
x_Estate = truncated_Estate_fingerprints(list(data['Mols']))
x_combined = np.concatenate((x_SOB, x_Estate), axis=1)
y = data['HOF(kJ/mol)'].values

np.savez('trade_off.npz', x_data=x_combined ,y_data=y)
data = np.load('trade_off.npz')
X = data['x_data']
y = data['y_data']

In [ ]:
iteration_time=[]

for i in range(200):
    print(i)
    round=0
    largest_y=np.where(y == np.max(y))   
    largesty_index=largest_y[0][0]
    
    while True:
        M = 20                  
        dataset_number = 88
        N = range(dataset_number)
        n = random.sample(N,M)
        
        if largesty_index in n:     
            continue
        else:
            train_y = y[n]   
            train_x = X[n]
            train_y = np.array(train_y)  
            break

    test_x = []
    test_y = []
    for i in range(dataset_number):
        if i not in n:
            x_ = X[i]
            test_x.append(x_)
            y_ = y[i]
            test_y.append(y_)

    GSmodel = GridSearchCV(SVR(kernel='linear'), cv=5, param_grid={"C": np.logspace(-1, 4, 20), "epsilon": np.logspace(-2, 2, 20)})
    GSmodel = GSmodel.fit(train_x,train_y)
    model = GSmodel.best_estimator_
    
    while True:
        result=[]
        for i in range(1000):
            bs_sample = np.random.choice(range(len(train_x)), size=len(train_x))
            x_bs = np.array(train_x)[bs_sample]
            y_bs = train_y[bs_sample]
            model.fit(x_bs,y_bs)
            y_pre = model.predict(test_x)
            result.append(y_pre)

        means = np.mean(result,axis=0)
        variance = np.var(result,axis=0)
        stds = np.std(result,axis=0)

        z = (means-np.max(train_y))/stds
        sdfs = norm.pdf(z)
        cdfs = norm.cdf(z)
        E = stds*(sdfs + z*cdfs)
        E_2 = sorted(E)
        item1 = np.where(E == E_2[-1])
        item1_index=item1[0][0]
        round +=1
        
        if test_y[item1_index] == np.max(y):
            iteration_time.append(round)
            break
        else:
            train_x = np.append(train_x, [test_x[item1_index]],axis = 0) 
            train_y = np.append(train_y, [test_y[item1_index]],axis = 0) 
            test_x = np.delete(test_x,item1_index,axis = 0) 
            test_y = np.delete(test_y,item1_index,axis = 0) 
            continue

In [ ]:
print(len(iteration_time))
means = np.mean(iteration_time,axis=0)
print(means)